In [4]:
import numpy
import pandas as pd
dataframe = pd.read_csv("tese-regression-model-v1\dataset_final.csv")
dataset = dataframe.values

#dataframe

In [5]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

X = dataset[:,35:39]
y = dataset[:,0:35]

poly = PolynomialFeatures(2)
X_poly = poly.fit_transform(X) #we now have a feature vector with 15 rows instead of only 4

standard_scaler = StandardScaler()
x_scaled = standard_scaler.fit_transform(X_poly)

scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y)

In [6]:
import keras.backend as K
import tensorflow as tf

def custom_loss(y_true, y_pred):
    
    percentage = 0.3
    
    pred_class = y_pred[..., 0:2]
    true_class = tf.argmax(y_true[..., 0:2], -1)
    loss_class = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=true_class, logits=pred_class)
    
    loss_reg1 = K.mean(K.square(y_pred[...,2:17] - y_true[...,2:17]), axis=-1)
    loss_reg2 = K.mean(K.square(y_pred[...,17:35] - y_true[...,17:35]), axis=-1)
    
    loss = percentage*loss_class + (1-percentage)*(loss_reg1*y_true[..., 0] + loss_reg2*y_true[..., 1])
    
    return loss

Using TensorFlow backend.


In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense    
from keras import regularizers

def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(15, input_dim=15, kernel_initializer='normal',  activation='relu'))
    model.add(Dense(35, kernel_initializer='normal'))
    # Compile model
    model.compile(loss=custom_loss, optimizer='adam') 
    
    return model

In [8]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from datetime import datetime

X_train, X_test, y_train, y_test = train_test_split(x_scaled, y,
                                                    test_size = 0.2,
                                                    random_state = seed)

model = baseline_model()

#Starting Time
start_time = datetime.now()
print('Start Time:', start_time)

history = model.fit(x_scaled, 
                    y, 
                    validation_split = 0.33,
                    epochs = 500, 
                    batch_size= 128, 
                    verbose = 0)

stop_time = datetime.now()

elapsed_time = stop_time - start_time
print('Elapsed Time:', elapsed_time) 

Start Time: 2018-02-21 20:12:03.323548
Elapsed Time: 0:00:38.493714


In [9]:
y_proba = model.predict(x_scaled)
print(y_proba)

[[  6.67342186e+00  -8.08210564e+00  -6.26647845e-03 ...,   5.52735031e-01
   -2.49085464e-02   3.49069446e-01]
 [  6.04120016e+00  -7.30890465e+00  -1.28443278e-02 ...,   5.84301114e-01
   -3.03126611e-02   3.72267127e-01]
 [  5.74679470e+00  -6.95585012e+00  -1.08176507e-02 ...,   5.60953259e-01
   -3.26927193e-02   3.55119079e-01]
 ..., 
 [ -5.65707731e+00   5.23779011e+00   1.38704777e-01 ...,   1.51706308e-05
   -1.10570818e-06   1.88238948e-04]
 [ -5.85119915e+00   5.43547153e+00   1.30306870e-01 ...,   1.52274033e-05
   -1.27554722e-06   2.17166002e-04]
 [ -5.52184153e+00   5.14607430e+00   1.71691969e-01 ...,   1.43405805e-05
   -6.98571966e-07   2.15710446e-04]]


In [10]:
y_classes = y_proba[...,:2].argmax(axis=-1)
print(y_classes)

[0 0 0 ..., 1 1 1]


In [11]:
numpy.set_printoptions(threshold=numpy.nan)
sess = tf.InteractiveSession()
a = tf.nn.softmax(logits=y_proba[...,:2], dim=-1)
a.eval()

array([[  9.99999642e-01,   3.90621068e-07],
       [  9.99998450e-01,   1.59265710e-06],
       [  9.99996901e-01,   3.04305854e-06],
       [  9.99999642e-01,   4.15380384e-07],
       [  9.99996662e-01,   3.30455009e-06],
       [  1.00000000e+00,   8.81758833e-11],
       [  9.99998093e-01,   1.89376829e-06],
       [  9.99833584e-01,   1.66396974e-04],
       [  9.99999046e-01,   9.21067340e-07],
       [  9.99999762e-01,   2.53503714e-07],
       [  9.99999404e-01,   5.72526005e-07],
       [  1.00000000e+00,   1.02919984e-08],
       [  1.00000000e+00,   1.17868271e-09],
       [  1.00000000e+00,   1.64847869e-08],
       [  1.00000000e+00,   4.40033041e-08],
       [  1.00000000e+00,   7.90301780e-09],
       [  1.00000000e+00,   1.93205523e-08],
       [  1.00000000e+00,   3.67424935e-09],
       [  1.00000000e+00,   1.38207401e-08],
       [  1.00000000e+00,   7.74157627e-09],
       [  1.00000000e+00,   5.14879195e-10],
       [  1.00000000e+00,   3.34515029e-08],
       [  

In [15]:
print(y[730])
print(y_proba[730])

[  1.00000000e+00   0.00000000e+00   1.50000000e-05   7.09000000e-05
   1.69000000e-05   7.00000000e-05   5.37000000e-06   2.67000000e-06
   8.77000000e-06   5.52000000e-06   3.00000000e+00   1.10000000e+01
   2.90000000e+01   2.00000000e+02   1.65000000e+02   5.66000000e-05
   1.98000000e+02   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00]
[  1.17141886e+01  -1.41703358e+01  -4.25553694e-03   5.48212603e-03
  -2.74118036e-04  -5.69065660e-03  -8.05155002e-03   3.57858464e-03
  -4.49683145e-03   7.76178017e-03   3.14413929e+00   9.67462730e+00
   5.03349495e+01   1.24316330e+02   1.29362152e+02   6.03023916e-03
   1.50813416e+02   8.63834858e-01   2.34121084e-01   7.56691992e-01
   9.46283266e-02  -5.85585833e-01   1.04982466e-0